# Исследование надежности заемщиков


**Цель проекта:**
* Разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


## Откроем таблицу и изучим общую информацию о данных

In [1]:
import pandas as pd

data = pd.read_csv('C:\\StudentYandex\\Projekts\\Python\\Projekt2Credit\\data.csv')


In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

Выведем количество пропущенных значений для каждого столбца:

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` заполним медианным доходом среди всех записей с тем же типом.

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными.

In [6]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится нам для исследования.

Выведем перечень уникальных значений столбца `children`.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения.

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный.

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.

In [14]:
data['education'] = data['education'].str.lower()

Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Посмотрим на уникальные цели взятия кредита из столбца `purpose`.

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Исследуем данные

#### Посмотрим есть ли зависимость между количеством детей и возвратом кредита в срок:

In [22]:
data.pivot_table(index = 'children', values = 'debt', aggfunc = ['count','sum','mean'])
# Выводим сводную таблицу, где видим 3 столбца и 6 строк(наличие/количество детей в семье), 
# в Первом столбце общее число заемщиков, 
# во Втором - Заемщики допустившие просрочку по кредиту, 
# в Третьем - доля Заемщиков допустивших просрочку, от общего числа взявших кредит.  


,count,sum,mean
,debt,debt,debt
children,,,
0,14091,1063,0.075438
1,4808,444,0.092346
2,2052,194,0.094542
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


**Вывод:** 

Данные показывают нам зависимость между количеством детей и возвратом кредита в срок:
* Заемщики без детей (их всех больше), но и доля не возврата кредита составляет 7,5 %;
* Заемщики у которых от одного до четырех детей, чаще допускают посроченную задолженность, чем Клиенты без детей;
* Можно было предположить что Заемщики с пятью детьми - возвращают кредиты в срок. Но в нашей выборке таких людей всего 9, всязи с этим нет оснований для подтверждения этой гипотезы.


#### Посмотрим есть ли зависимость между семейным положением и возвратом кредита в срок:

In [23]:
data.pivot_table(index = 'family_status', values = 'debt', aggfunc = ['count','sum','mean'])# Ваш код будет здесь. Вы можете создавать новые ячейки.

,count,sum,mean
,debt,debt,debt
family_status,,,
Не женат / не замужем,2796,273,0.097639
в разводе,1189,84,0.070648
вдовец / вдова,951,63,0.066246
гражданский брак,4134,385,0.093130
женат / замужем,12261,927,0.075606


**Вывод:** 

Данные показывают нам зависимость между семейным положением и возвратом кредита в срок:
* Доля не возврата кредита больше у Заемщиков, которые не состоят в браке или находятся в гражданском браке;
* Заемщики, состоящие в браке, либо находящиеся в разводе допускают просрочки реже чем Клиенты которые не состоят в браке или находятся в гражданском браке;
* Самыми благонадежными плательщиками являются Заемщики со статусом вдовец/вдова.

Кредитному отделу следует с осторожностью относиться к клиентам, которые не состоят или не состояли в браке. Согласно данным такие клиенты менее ответственные. Получается, что узаконенные отношения (включая и бывшие отношения) ведут к более ответственному отношению к своим обязательствам. Ну вроде как совпадает со здравым смыслом - семейные люди более скрупулёзно берегут свой семейное спокойствие и несут за него ответственность.

#### Проверим есть ли зависимость между уровнем дохода и возвратом кредита в срок:

In [24]:
data.pivot_table(index = 'total_income_category', values = 'debt', aggfunc = ['count','sum','mean'])# Ваш код будет здесь. Вы можете создавать новые ячейки.

,count,sum,mean
,debt,debt,debt
total_income_category,,,
A,25,2,0.080000
B,5014,354,0.070602
C,15921,1353,0.084982
D,349,21,0.060172
E,22,2,0.090909


**Вывод:** 

Данные показывают нам зависимость между уровнем дохода и возвратом кредита в срок:
* Клиенты с уровнем дохода от 30000 - 50000 являются самыми благонадежными;
* Самая большая доля не возврата кредита у заемщиков с наименьшим доходом.

Исходя из здравого смысла логично предположить, что чаще всего должниками становятся люди с небольшим доходом, которым финансовые проблемы не позволяют вовремя осуществлять платежи. Но глядя на наши результаты лично мой вывод таков: выборка по заемщикам, как с наименьшим доходом, так и с наибольшим крайне недостаточна для формирования однозначных выводов. Можно думать, что E-категория самая низкооплачиваемая не может корректно распоряжаться деньгами и «влезает» в кредиты, но тогда неясно почему у А-категории также достаточно высокий процент задолженности. В общем вывод по группам А и Е неочевиден, мало данных.

Если же взять для анализа только группы В, С и D, то по полученным данным не наблюдаем зависимости "те, кто получает больше ЗП, лучше возвращает кредит». В целом это логично, так как материальный достаток - не единственная переменная, которая влияет на качество кредитной истории.

Однако 2174 пропуска в данных столбца "total_income" ставят под сомнение полученный нами результат. Сводная таблица (или группировка) с данными о зависимости между уровнем дохода и возвратом кредита в срок возможно могла бы выглядеть иначе, если бы не пропущенные данные.

#### Посмотрим как разные цели кредита влияют на его возврат в срок:

In [25]:
data.pivot_table(index = 'purpose_category', values = 'debt', aggfunc = ['count','sum','mean'])# Ваш код будет здесь. Вы можете создавать новые ячейки.

,count,sum,mean
,debt,debt,debt
purpose_category,,,
операции с автомобилем,4279,400,0.093480
операции с недвижимостью,10751,780,0.072551
получение образования,3988,369,0.092528
проведение свадьбы,2313,183,0.079118


**Вывод:** 

Данные показывают нам как разные цели кредита влияют на его возврат в срок:
* Кредиты взятые на недвижимость и свадьбу имеют меньшую долю просрочки;
* А вот кредиты взятые на автомобиль и образование оплачиваются почти на 2% хуже. 

Машины люди покупают в кредит гораздо дорогие, нежели могут соответствовать им своим доходом (ну кому не хочется красивую иномарку). А вот рисковать ипотечным жильём люди не хотят. Интересно конечно, что с образованием не так - как правило такие кредиты берут люди мотивированные, имеющие цель впереди, да и суммы таких кредитов судя по рынку онлайн образования не так велики. Ну и получается, что ещё более мотивированные люди берут деньги на организацию своей свадьбы)

Причина пропусков данных Технологическая. Т.к. один и тот же пул клиентов остался без Дохода(total_income) и Стажа(days_employed)

в наших данных пропуски в столбцах "трудовой стаж" и "ежемесячный доход" абсолютно идентичны. Количество строк с отсутствующими данными в обоих указанных столбцах совпадает и равно 2174. Маловероятно, что специалисты по выдаче кредитов, заполнив все без исключения ячейки других столбцов по каждому кредитополучателю, оставляли незаполненными именно два показателя, и сделали это ровно 2174 раза в каждом столбце. Даже не проводя проверку, во всех ли случаях незаполненные ячейки находятся в одной и то же строке с высокой долей вероятности можно утверждать, что произошла техническая ошибка при формировании таблицы с исходными данными (либо некий сбой во время выгрузки данных на анализ, либо при объединении данных в единый файл, который также должен был передаваться на анализ, либо эти данные не собирались).

Используем "Медиану" т.к. "Среднее" сильно поднимет планку вверх, из за сильной разницы в значениях min и max значений в таблице. Что в свою очередь приведет к сильному искажению данных вверх. Медиана зачастую даёт более правдивую картину. 

# Общий вывод:
Мы провели исследование как различные факторы влияют на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов. Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Перед проведением исследования выдвинули четыре гипотезы:

1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

**Гипотеза 1 (Количество детей):**

* Заемщики без детей (их всех больше), но и доля не возврата кредита составляет 7,5 %;
* Заемщики у которых от одного до четырех детей, чаще допускают посроченную задолженность, чем Клиенты без детей ~ на 2%;
* Заемщики с пятью детьми - возвращают кредиты в срок. (Но для подтверждения данного утверждения требуется больше данных)

Кредитополучатели без детей являются более ответственными, чем клиенты, у которых есть дети. По всей видимости, наличие ребенка накладывает на клиента дополнительные финансовое бремя (обязательства) и клиенты либо менее ответственно относятся к своим кредитным обязательствам, либо не имеют возможности в срок оплачивать обязательства.

**Гипотеза 2 (Семейное положение):**

* Доля не возврата кредита больше у Заемщиков, которые не состоят в браке или находятся в гражданском браке, в среднем 9,5 %;
* Заемщики, состоящие в браке, либо находящиеся в разводе допускают просрочки реже чем Клиенты которые не состоят в браке или находятся в гражданском браке ~ на 2 %;
* Самыми благонадежными плательщиками являются Заемщики со статусом вдовец/вдова - 6,6 %.

Кредитному отделу следует с осторожностью относиться к клиентам, которые не состоят или не состояли в браке. Согласно данным такие клиенты менее ответственные. Получается, что узаконенные отношения (включая и бывшие отношения) ведут к более ответственному отношению к своим обязательствам.

**Гипотеза 3 (Уровень дохода):**

Для проведенния данного исследования, разбили клиентов на катогории:
0–30000 — 'E';
30001–50000 — 'D';
50001–200000 — 'C';
200001–1000000 — 'B';
1000001 и выше — 'A'.

* Клиенты с уровнем дохода 'D' являются самыми благонадежными. Доля не возврата кредитов - 6 %;
* Самая большая доля не возврата кредита у заемщиков с доходом 'E' - 9 %.(Но для подтверждения данного утверждения требуется больше данных)

Можно думать, что E-категория самая низкооплачиваемая не может корректно распоряжаться деньгами и «влезает» в кредиты, но тогда неясно почему у А-категории также достаточно высокий процент задолженности (8%). В общем вывод по группам А и Е неочевиден, мало данных.
Если же взять для анализа только группы В, С и D, то по полученным данным не наблюдаем зависимости "те, кто получает больше, лучше возвращает кредит».

**Гипотеза 4 (Цель кредита):**

* Кредиты взятые на недвижимость и свадьбу имеют меньшую долю просрочки ~ 7,5 %;
* А вот кредиты взятые на автомобиль и образование оплачиваются почти на 2 % хуже.

Машины люди покупают в кредит гораздо дорогие, нежели могут соответствовать им своим доходом. А вот рисковать ипотечным жильём люди не хотят.


**Общие рекомендации:**
1. Гипотезы *Семейное положение* и *Цель кредита* полностью подтвердились. Рекомендую специалистам использовать эти зависимости для построения скорринговой модели при кредитовании клиентов банка.
2. Гипотезы *Количество детей* и *Уровень дохода* по предоставленной статистике не возможно опровергнуть или подтвердить, необходимо более полную выборку для подтверждения данных утверждений.
